In [52]:
import os
import requests
import pandas as pd
import gspread

from dotenv import load_dotenv
load_dotenv()



True

In [122]:
# 1. забираем лиды из Google Forms

FORM_WORKSHEET = 'Сборщик лидов (Responses)'
REPORT_WORKSHEET = 'Leads Report'

telegram_token = os.getenv('telegram_token')
telegram_chat_id = os.getenv('telegram_chat_id')

In [123]:
gc = gspread.service_account('credentials.json')
worksheet = gc.open(FORM_WORKSHEET).sheet1
form_leads = pd.DataFrame(worksheet.get_all_records())

form_leads.columns = ['timestamp', 'email', 'company_size', 'city']
form_leads['timestamp'] = pd.to_datetime(form_leads['timestamp'])
form_leads['dt'] = form_leads['timestamp'].dt.date
form_leads['year_month'] = form_leads['timestamp'].dt.to_period('M')

In [124]:
# 2. скачиваем текущие лиды из отчета

worksheet = gc.open(REPORT_WORKSHEET).worksheet('Leads')
report_leads = pd.DataFrame(worksheet.get_all_records())



In [125]:
# 3. сравниваем форму и отчет, находим новые лиды 

merged_leads = form_leads.merge(
    report_leads, 
    how='left', 
    on='email', 
    suffixes=('', '_drop'),
    indicator=True)

new_leads = merged_leads[merged_leads['_merge'] == 'left_only'][form_leads.columns]
new_leads


,timestamp,email,company_size,city,dt,year_month


In [126]:
# 4. отправляем сообщение в телеграм

def leads_to_text(df):
    output = []
    for df_line in df.iterrows():
        row = df_line[1]
        output.append(f"{row['email']} - {row['company_size']} - {row['city']}")
    return '\n'.join(output)

message = 'New Leads:\n' + leads_to_text(new_leads)

url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
payload = {
    'chat_id': telegram_chat_id,
    'text': message
}

response = requests.post(url, data=payload)
response



<Response [200]>

In [127]:
# 2. загружаем новые лиды в отчет

worksheet = gc.open(REPORT_WORKSHEET).worksheet('Leads')
worksheet.update([form_leads.astype('str').columns.values.tolist()] + form_leads.astype('str').values.tolist())

/var/folders/s_/t8ytmm9940j564xd698kgmm00000gn/T/ipykernel_76994/2536004802.py:4: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([form_leads.astype('str').columns.values.tolist()] + form_leads.astype('str').values.tolist())


{'spreadsheetId': '1ApyyiCups2STIPdFjdesxabwHuKuZ9cnXGuR3ik2TOk',
 'updatedRange': 'Leads!A1:F5',
 'updatedRows': 5,
 'updatedColumns': 6,
 'updatedCells': 30}

<Response [200]>